# QR Algorithm

Ci sono delle proprietà:

- Dato che gli eigenvalue di una matrice tirangolare sono gli elementi sulla diagonale, questo algoritmo permette di trovarli.
- Similarity transformation preserva gli eigenvalue.

La matrice A viene **decomposta** in A = QR dove Q è ortogonale e R è triangolare superiore.

In [3]:
import numpy as np
from itertools import product
from math import copysign, hypot, sqrt


def print_matrix(matrix):
    s = [[str(e) for e in row] for row in matrix]
    lens = [max(map(len, col)) for col in zip(*s)]
    fmt = '\t'.join('{{:{}}}'.format(x) for x in lens)
    table = [fmt.format(*row) for row in s]
    print('\n'.join(table))
    return

## Gram-Schmidt
Come prima implementazione proviamo con l'algoritmo di **Gram-Schmidt**


In [ ]:
def qr_decomp_gram_schmidt(A):
    m, n = A.shape
    rank = np.linalg.matrix_rank(A)

    if rank < n:
        print(f"Il rango della matrice è {rank} il quale è inferiore del numero delle colonne {n}!")
    
    Q = np.zeros((m, n))
    
    for i, column in enumerate(A.T):
        Q[:,i] = column

        for prev in Q.T[:i]:
            Q[:,i] -= (prev @ column) / (prev @ prev) * prev
    
    Q /= np.linalg.norm(Q,axis=0)
    R = Q.T @ A

    return Q, R

In [ ]:
# Tests

#TODO: rendere i test non dipendenti da una sola funzione

def test_orthonormality(A):
    Q, _ = qr_decomp_gram_schmidt(A)
    I = Q.T @ Q

    assert len(set(I.shape)) == 1, "L'identità computata deve essere una matrice quadrata!"
    assert np.allclose(I, np.eye(I.shape[0])), "L'identità  computata deve essere una matrice identità!"

def test_upper_triangular(A):
    _, R = qr_decomp_gram_schmidt(A)

    assert len(set(R.shape)) == 1, "La matrice triangolare superiore computata dovrebbe essere una matrice quadrata!"

    for i, row in enumerate(R):
        assert np.allclose(row[:i], 0), f"La riga {i} della matrice triangolare superiore calcolata non è giusta!"

def test_multiplication(A):
    Q,R = qr_decomp_gram_schmidt(A)
    
    assert np.allclose(Q @ R, A), "La moltiplicazione fra le matrici ottenute non restituisce la matrice originaria!"

In [ ]:
test_matricies = (
    np.random.random((2, 2)),
    np.random.random((5, 2)),
    np.random.random((5, 5)),
    np.random.random((9, 8)),
)

tests = (
    test_orthonormality,
    test_upper_triangular,
    test_multiplication,
)

for test, matrix in product(tests, test_matricies):
    test(matrix)

print("tutti i test sono superati")

# Givens Rotation
proviamo ad implementare le givens rotation per migliorare la velocità di esecuzione.

In [ ]:
def qr_decomp_givens_rotation(A):

    (num_rows, num_cols) = np.shape(A)
    if num_rows == num_cols: 
        # Initialize Q,R
        # Q = orthogonal matrix
        # R =  upper triangular matrix
        Q = np.identity(num_rows)
        R = np.copy(A)

        # Iterate over lower triangular matrix.
        (rows, cols) = np.tril_indices(num_rows, -1, num_cols)
        #print(rows, cols)
        for (row, col) in zip(rows, cols):

            # Compute Givens rotation matrix and
            # zero-out lower triangular matrix entries.
            if R[row, col] != 0:
                (c, s) = Givens_Rotation_Matrix_Entries(R[col, col], R[row, col])

                G = np.identity(num_rows)
                G[[col, row], [col, row]] = c
                G[row, col] = s
                G[col, row] = -s

                R = np.dot(G, R)
                Q = np.dot(Q, G.T)
                #R = G @ R
                #Q = Q @ G.T
        return Q, R
    else:
        Q = np.eye(num_rows)
        R = np.copy(A)

        rows, cols = np.tril_indices(num_rows, -1, num_cols)
        for (row, col) in zip(rows, cols):
            # If the subdiagonal element is nonzero, then compute the nonzero 
            # components of the rotation matrix
            if R[row, col] != 0:
                r = np.sqrt(R[col, col]**2 + R[row, col]**2)
                c, s = R[col, col]/r, -R[row, col]/r

                # The rotation matrix is highly discharged, so it makes no sense 
                # to calculate the total matrix product
                R[col], R[row] = R[col]*c + R[row]*(-s), R[col]*s + R[row]*c
                Q[:, col], Q[:, row] = Q[:, col]*c + Q[:, row]*(-s), Q[:, col]*s + Q[:, row]*c

        return Q[:, :num_cols], R[:num_cols]



##### Compute matrix entries for Givens rotation. #####

def Givens_Rotation_Matrix_Entries(a, b):
    r = hypot(a, b)
    c = a/r
    s = -b/r

    return (c, s)

In [ ]:
def test_orthonormality(A):
    Q, _ = qr_decomp_givens_rotation(A)
    I = Q.T @ Q

    assert len(set(I.shape)) == 1, "L'identità computata deve essere una matrice quadrata!"
    assert np.allclose(I, np.eye(I.shape[0])), "L'identità  computata deve essere una matrice identità!"

def test_upper_triangular(A):
    _, R = qr_decomp_givens_rotation(A)

    assert len(set(R.shape)) == 1, "La matrice triangolare superiore computata dovrebbe essere una matrice quadrata!"

    for i, row in enumerate(R):
        assert np.allclose(row[:i], 0), f"La riga {i} della matrice triangolare superiore calcolata non è giusta!"

def test_multiplication(A):
    Q, R = qr_decomp_givens_rotation(A)
    
    assert np.allclose(Q @ R, A), "La moltiplicazione fra le matrici ottenute non restituisce la matrice originaria!"

In [ ]:
test_matricies = (
    np.random.random((3, 3)),
    np.random.random((4, 5)),
    np.random.random((5, 5)),
    np.random.random((9, 8)),
)

tests = (
    test_orthonormality,
    test_multiplication,
)

for test, matrix in product(tests, test_matricies):
    test(matrix)

print("tutti i test sono superati")

In [ ]:
A = np.random.random((4, 4))

Q, R = qr_decomp_givens_rotation(A)

print(A)
print(Q)
print(R)

print(Q @ R)

# Shifted QR Algorithm

Questo approccio ci consente di ridurre considerevolmente il tempo di esecuzione dell'algoritmo QR, ma per poterlo utilizzare è necessario ridurre la matrice di partenza in forma di Hessenberg

## Riduzione in forma di Hessenberg
Per ridurre la matrice di partenza A in forma di Hessenberg utilizzeremo le Givens Rotation, per azzerare gli elementi sotto la prima sottodiagonale.
Iteriamo lungo le colonne a partire dall'ultimo elemento, così da poter scegliere come pivot l'elemento immediatamente sopra.

In [137]:
# TODO da controllare se funziona per matrici non quadrate, e in caso implementare una versione funzionante.

def Givens_Rotation_Matrix_Entries(a, b):
    r = hypot(a, b)
    c = a/r
    s = b/r    
    return c.astype(complex), s.astype(complex)

def Hessenberg_reduction(A):
    # mi trovo gli indici della sottodiagonale corrispondente agli elementi da azzerare
    (num_rows, num_cols) = np.shape(A)
    # Q ci serve per calcolarci la forma di Schur
    Q = np.identity(num_rows)
    Q.astype(complex)
    #itero sugli indici appena trovati 
    for col in range(num_cols-2):
        for row in reversed(range(2+col,num_cols)):
            # trova gli indici della matrice di givens unando come pivot [col, col](sempre != 0)
            c, s = Givens_Rotation_Matrix_Entries(A[row-1, col], A[row, col])
            #si costruisce la matrice di givens partendo dall'identità
            G = np.identity(num_rows)
            G.astype(complex)
            #assegna gli indici trovati precedentemente alla matrice di givens
            G[row-1, row-1] = c
            G[row, row] = c
            G[row, row-1] = -s
            G[row-1, row] = s
            # annichilisce l'elemento in posizione A[row, col]
            A = np.dot(G,A)
            Q = np.dot(Q, G.T)

    return A, Q

In [9]:
A = np.random.random((8, 8))
(A, Q) = Hessenberg_reduction(A)
print_matrix(A)

0.8168909564405686     	0.7318280307193928     	0.9386615793528416     	0.678916925485609     	0.8581252290789727    	0.7672068513251721     	0.0255518945888652 	0.12201551552717094 
1.2367108616570899     	1.2116713450294945     	0.7519014384800832     	1.3624561178865666    	1.2426457192236942    	1.4390966852650604     	0.8188141331458173 	1.150232152770037   
3.4660535367960294e-17 	1.023259403726648      	0.20574808604594536    	0.21882405216794584   	0.29212779000719863   	0.6542372144542663     	0.4349099846615021 	0.34295878214957504 
2.5874069259601356e-17 	-2.6416855723989436e-17	0.5447237064451343     	0.15658546611369795   	0.5860251714907346    	0.4399400728792093     	0.6290116294962129 	0.19089312485011098 
-1.316203658182628e-17 	5.19819791962338e-18   	3.7873195336259453e-20 	0.47526802004383284   	0.60017653494278      	0.4722683251325442     	0.13599040994568548	0.13813946349196632 
-2.7049777034544792e-17	-3.055642893821751e-17 	8.103813365343122e-18  	7.33483809509

## Shifted QR
Ora che abbiamo una matrice in forma di Hessenberg possiamo passare all'implementazione dello Shifted QR algorithm, l'aumento di velocità dipende dal fatto che, se si parte da una matrice in forma di Hessenberg allora sono necessarie solo O(n^2) operazioni.

In particolare noi utilizzeremo una single shift strategy con i Rayleigh quotient shift

In [154]:
def shifted_QR(A):
    H, _ = Hessenberg_reduction(A)
    H.astype(complex)
    (num_rows, num_cols) = np.shape(H)
   
    #U = np.eye(num_rows)
    for m in range(num_rows-1, 2, -1):
        while (H[m,m-1] > 1.e-30):
            #print("Primo H[m,m]", H[m,m])
            #print("Prima")
            #print_matrix((H[m,m] * np.eye(num_rows)))
            #print("Dopo")
            #print_matrix((H - (H[m,m] * np.eye(num_rows))))
            #return
            Q, R = np.linalg.qr(H - (H[m,m] * np.eye(num_rows)))
            H = (R @ Q) + (H[m,m] * np.eye(num_rows))
            #U = U @ Q
            #print("Secondo H[m,m]", H[m,m])
    #return H, U
    return H

In [164]:
A = np.random.random((5, 5))
A = A.astype(complex)
H = shifted_QR(A)
print_matrix(H)

(2.3285102564094955+0j)    	(-0.4180674056830747+0j)    	(1.1449222985749807+0j)     	(1.2823976256737648+0j)  	(0.3537597911453401+0j)  
(-0.036043060206582024+0j) 	(-0.7445925075486449+0j)    	(-0.024456920733666062+0j)  	(0.2300592794927976+0j)  	(-0.0876096285728312+0j) 
(6.914884285654394e-20+0j) 	(-0.07135923311481643+0j)   	(0.7075085815083706+0j)     	(-0.21253474088739793+0j)	(-0.37722928174778475+0j)
(-9.008340872233941e-23+0j)	(-1.7086392679762232e-21+0j)	(-0.00019246860262163082+0j)	(0.17050587488225627+0j) 	(0.08106495208094557+0j) 
(-5.949558075457681e-23+0j)	(-1.1294011928344133e-21+0j)	(3.9584318332761753e-20+0j) 	(-0.19135598110937682+0j)	(0.3600017514736135+0j)  


C:\Users\tomma\AppData\Local\Temp\ipykernel_11664\1503993989.py:4: ComplexWarning: Casting complex values to real discards the imaginary part
  r = hypot(a, b)
C:\Users\tomma\AppData\Local\Temp\ipykernel_11664\1503993989.py:24: ComplexWarning: Casting complex values to real discards the imaginary part
  G[row-1, row-1] = c
C:\Users\tomma\AppData\Local\Temp\ipykernel_11664\1503993989.py:25: ComplexWarning: Casting complex values to real discards the imaginary part
  G[row, row] = c
C:\Users\tomma\AppData\Local\Temp\ipykernel_11664\1503993989.py:26: ComplexWarning: Casting complex values to real discards the imaginary part
  G[row, row-1] = -s
C:\Users\tomma\AppData\Local\Temp\ipykernel_11664\1503993989.py:27: ComplexWarning: Casting complex values to real discards the imaginary part
  G[row-1, row] = s
